


### 南京大学计算传播学系列课程
***
***
# 《计算传播学的编程基础》
***
***


王成军 

wangchengjun@nju.edu.cn

<img align="left" width = "500px" style="padding-right:10px;" src="figures/header2.png">


<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="figures/PDSH-cover-small.png">
*This notebook contains an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*

*The text is released under the [CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode), and code is released under the [MIT license](https://opensource.org/licenses/MIT). If you find this content useful, please consider supporting the work by [buying the book](http://shop.oreilly.com/product/0636920034919.do)!*

<!--NAVIGATION-->
< [Errors and Debugging](01.06-Errors-and-Debugging.ipynb) | [Contents](Index.ipynb) | [More IPython Resources](01.08-More-IPython-Resources.ipynb) >

# Profiling and Timing Code

In the process of developing code and creating data processing pipelines, there are often trade-offs you can make between various implementations.

- Early in developing your algorithm, it can be counterproductive to worry about such things. 

As Donald Knuth famously quipped, 

> "We should forget about small efficiencies, say about 97% of the time: premature optimization is the root of all evil."

But once you have your code working, it can be useful to dig into its efficiency a bit.

- it's useful to check the execution time of a given command or set of commands; 
- it's useful to dig into a multiline process and determine where the bottleneck lies in some complicated series of operations.

IPython provides access to a wide array of functionality for this kind of timing and profiling of code.


Here we'll discuss the following IPython magic commands:

- ``%time``: Time the execution of a single statement
- ``%timeit``: Time repeated execution of a single statement for more accuracy
- ``%prun``: Run code with the profiler
- ``%lprun``: Run code with the line-by-line profiler
- ``%memit``: Measure the memory use of a single statement
- ``%mprun``: Run code with the line-by-line memory profiler

The last four commands are **NOT** bundled with IPython
- you'll need to get the **``line_profiler``** and **``memory_profiler``** extensions

## Timing Code Snippets: ``%timeit`` and ``%time``

- ``%timeit`` line-magic 
- ``%%timeit`` cell-magic 

See also the Introduction to magic functions in [IPython Magic Commands](01.03-Magic-Commands.ipynb);


In [1]:
%timeit sum(range(100))

100000 loops, best of 3: 1.54 µs per loop


## Note

- Because this operation is so fast, ``%timeit`` automatically does a large number of **repetitions**.
- For slower commands, ``%timeit`` will automatically adjust and perform fewer repetitions

In [2]:
%%timeit
total = 0
for i in range(1000):
    for j in range(1000):
        total += i * (-1) ** j
print(total)

0
0
0
0
1 loop, best of 3: 340 ms per loop


Sometimes repeating an operation is not the best option.

### Example

> Sorting a pre-sorted list is much faster than sorting an unsorted list

If we have a list that we'd like to sort, we might be misled by a repeated operation.


In [3]:
import random
L = [random.random() for i in range(100000)]
%timeit L.sort()

The slowest run took 23.26 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 1.56 ms per loop


For this, the **``%time``** magic function may be a better choice. 

- It also is a good choice for longer-running commands, 
- when short, system-related delays are unlikely to affect the result.

Let's time the sorting of an unsorted and a presorted list:

In [4]:
import random
L = [random.random() for i in range(100000)]
print("sorting an unsorted list:")
%time L.sort()

sorting an unsorted list:
CPU times: user 37.7 ms, sys: 745 µs, total: 38.5 ms
Wall time: 38.6 ms


In [5]:
print("sorting an already sorted list:")
%time L.sort()

sorting an already sorted list:
CPU times: user 3.2 ms, sys: 8 µs, total: 3.21 ms
Wall time: 3.21 ms


### Notice 
- how much faster the presorted list is to sort
- how much longer the timing takes with ``%time`` versus ``%timeit``, even for the presorted list!

``%timeit`` does some clever things under the hood to prevent system calls from interfering with the timing.
- it prevents cleanup of unused Python objects (known as *garbage collection*) 


**``%%``** allows timing of multiline scripts:

In [6]:
%%time
total = 0
for i in range(1000):
    for j in range(1000):
        total += i * (-1) ** j

CPU times: user 451 ms, sys: 6.37 ms, total: 457 ms
Wall time: 460 ms


### More information 

- use the IPython help functionality
    - type **``%time?``** at the IPython prompt

In [8]:
%time?

## Profiling Full Scripts: ``%prun``

- A program is made of many single statements
- **timing these statements in context** is more important than timing them on their own.
- Using the magic function ``%prun``.

### Example 

We'll define a simple function that does some calculations:

In [32]:
def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
    return total

``>>`` and ``<<`` are the Right-Shift and Left-Shift bit-operators, i.e., they alter the binary representation of the number

In [26]:
bin(10) # 10 in binary

'0b1010'

In [27]:
10 >> 1 # Shifting all the bits to the right and discarding the rightmost one

5

In [28]:
bin(_) # 5 in binary - you can see the transformation clearly now

'0b101'

In [29]:
10 >> 2 # Shifting all the bits right by two and discarding the two-rightmost ones

2

In [30]:
bin(_)

'0b10'

Now we can call ``%prun`` with a function call to see the profiled results:

In [33]:
%prun sum_of_lists(1000000)

In the notebook, the output is printed to the pager, and looks something like this:

```
14 function calls in 0.714 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.599    0.120    0.599    0.120 <ipython-input-19>:4(<listcomp>)
        5    0.064    0.013    0.064    0.013 {built-in method sum}
        1    0.036    0.036    0.699    0.699 <ipython-input-19>:1(sum_of_lists)
        1    0.014    0.014    0.714    0.714 <string>:1(<module>)
        1    0.000    0.000    0.714    0.714 {built-in method exec}
```

 
- The bulk of execution time is in the list **comprehension** inside ``sum_of_lists``.
- Thus, we can think about improving the performance in the algorithm.

### More information on ``%prun``

> # ``%prun?``

## Line-By-Line Profiling with ``%lprun``

- line-by-line profile report.
- ``line_profiler`` package

Start by using Python's packaging tool, ``pip``, to install the ``line_profiler`` package:

```
$ pip install line_profiler
```

or


```
$ conda install line_profiler
```

In [35]:
%load_ext line_profiler

- ``%lprun`` command will do a line-by-line profiling of any function, 
- we need to tell it explicitly which functions we're interested in profiling

In [36]:
%lprun -f sum_of_lists sum_of_lists(5000)

As before, the notebook sends the result to the pager, but it looks something like this:

```
Timer unit: 1e-06 s

Total time: 0.009382 s
File: <ipython-input-19-fa2be176cc3e>
Function: sum_of_lists at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def sum_of_lists(N):
     2         1            2      2.0      0.0      total = 0
     3         6            8      1.3      0.1      for i in range(5):
     4         5         9001   1800.2     95.9          L = [j ^ (j >> i) for j in range(N)]
     5         5          371     74.2      4.0          total += sum(L)
     6         1            0      0.0      0.0      return total
```


- the time is reported in microseconds 
- we can see where the program is spending the most time.

To further modify the script and make it perform better.

### More information on ``%lprun``

> # type ``%lprun?`` 

## Profiling Memory Use: ``%memit`` and ``%mprun``

profiling the amount of memory an operation uses.

IPython extension, the ``memory_profiler``.

we start by ``pip``-installing the extension:

```
$ pip install memory_profiler
```

In [37]:
%load_ext memory_profiler

The memory profiler extension contains two useful magic functions: 
- the ``%memit`` magic
    - which offers a memory-measuring equivalent of ``%timeit``
- the ``%mprun`` function 
    - which offers a memory-measuring equivalent of ``%lprun``


In [38]:
%memit sum_of_lists(1000000)

peak memory: 126.23 MiB, increment: 45.83 MiB


We see that this function uses about 100 MB of memory.

For a line-by-line description of memory use, we can use the ``%mprun`` magic.

### Note

- this magic works only for functions defined in **separate modules** rather than the notebook itself, 
- so we'll start by using the ``%%file`` magic to create a simple module called ``mprun_demo.py``, 


In [39]:
%%file mprun_demo.py
def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
        del L # remove reference to L
    return total

Writing mprun_demo.py


We can now import the new version of this function and run the memory line profiler:

In [45]:
from mprun_demo import sum_of_lists
%mprun -f sum_of_lists sum_of_lists(100000)

('',)


The result, printed to the pager, gives us a summary of the memory use of the function, and looks something like this:
```
Filename: mprun_demo.py

Line #    Mem usage    Increment   Line Contents
================================================
     1    113.3 MiB    113.3 MiB   def sum_of_lists(N):
     2    113.3 MiB      0.0 MiB       total = 0
     3    114.1 MiB      0.0 MiB       for i in range(5):
     4    114.6 MiB -140977.6 MiB           L = [j ^ (j >> i) for j in range(N)]
     5    114.9 MiB      1.2 MiB           total += sum(L)
     6    114.1 MiB     -3.8 MiB           del L # remove reference to L
     7    114.1 MiB      0.0 MiB       return total
```

# More information on ``%memit`` and ``%mprun``

> # type ``%memit?``

<!--NAVIGATION-->
< [Errors and Debugging](01.06-Errors-and-Debugging.ipynb) | [Contents](Index.ipynb) | [More IPython Resources](01.08-More-IPython-Resources.ipynb) >